In [17]:
from __future__ import print_function, division
import fnmatch
import os
import random
import cPickle as pickle

#from envmap.environmentmap import EnvironmentMap
import numpy as np
from hdrio import imread, imsave
from scipy.ndimage.interpolation import zoom
from matplotlib import pyplot as plt

STACK_FILENAME = 'stack.exr'
TARGET_SIZE = (256, 256)


# Get the data
def getAvailableData(root_path, filename=STACK_FILENAME):
    matches = []
    for root, dirnames, filenames in os.walk(root_path):
        for filename in fnmatch.filter(filenames, filename):
            matches.append(os.path.join(root, filename))
    return matches


def generateLDRfromHDR(im_path, out_prefix):
    """Convert an HDR image into a clipped 0-255 value ("simulating" a camera)"""
    print('Processing: ', im_path)
    im = imread(im_path)
    
    # ROGNTUDJU, pourquoi c'est pas carre?
    h, w, c = im.shape
    im = im[:, w/2 - h/2:w/2 + h/2]
    
    valid = (im > 0) & (~np.isnan(im))
    im_median = np.median(im[valid])
    im_low = np.percentile(im[valid], 3)
    im_high = np.percentile(im[valid], 95)
    
    scales = (TARGET_SIZE[0]/im.shape[0], TARGET_SIZE[1]/im.shape[1])
    im = zoom(im, [scales[0], scales[1], 1])
    
    with open(out_prefix + "_hdr.pkl", 'wb') as fhdl:
        pickle.dump(im, fhdl, pickle.HIGHEST_PROTOCOL)
    imsave(out_prefix + '_hdr.exr', im)
    
    # 20th percentile -> value 5
    # 80th percentile -> value 250
    print("Ratio:", (im_high - im_low))
    ratio = im_high - im_low
    if ratio < 0.1:
        ratio = 0.1
    im_ldr = (im - im_low) * 250. / ratio + 5
    im_ldr = np.clip(im_ldr, 0, 255).astype('uint8')
    
    imsave(out_prefix + '_ldr.jpg', im_ldr)
    
    plt.figure()
    plt.subplot(1,2,1); plt.hist(im.ravel()[im.ravel()<im_high], 50)
    plt.subplot(1,2,2); plt.hist(im_ldr.ravel()[im_ldr.ravel()>0], 50)
    plt.savefig(out_prefix + 'debug.png')
    plt.close()

    
def main():
    im_paths = getAvailableData('/gel/rachmaninoff/data/pictures/master/skycam/')
    im_paths = random.sample(im_paths, 1000)
    for im_path in im_paths:
        root_path = os.path.dirname(im_path)
        out_path = os.path.join('data', "_".join(root_path.split(os.sep)[-2:]))
        try:
            generateLDRfromHDR(im_path, out_path)
        except Exception as e:
            print("Error happened:", e)


if __name__ == '__main__':
    main()
    

Processing:  /gel/rachmaninoff/data/pictures/master/skycam/20141127/131841/stack.exr
Ratio: 0.176681518555
Processing:  /gel/rachmaninoff/data/pictures/master/skycam/20141114/143737/stack.exr
Ratio: 0.191925048828
Processing:  /gel/rachmaninoff/data/pictures/master/skycam/20141102/144425/stack.exr
Ratio: 0.250289916992
Processing:  /gel/rachmaninoff/data/pictures/master/skycam/20131119/120224/stack.exr
Ratio: 0.02734375
Processing:  /gel/rachmaninoff/data/pictures/master/skycam/20141030/171626/stack.exr
Ratio: 0.00224304199219
Processing:  /gel/rachmaninoff/data/pictures/master/skycam/20141108/110025/stack.exr
Ratio: 0.327484130859
Processing:  /gel/rachmaninoff/data/pictures/master/skycam/20141005/144405/stack.exr
Ratio: 0.202011108398
Processing:  /gel/rachmaninoff/data/pictures/master/skycam/20130807/113309/stack.exr
Ratio: 0.191741943359
Processing:  /gel/rachmaninoff/data/pictures/master/skycam/20130624/140636/stack.exr
Ratio: 0.240524291992
Processing:  /gel/rachmaninoff/data/pic

/home/local/shared/anaconda/envs/yahognn/lib/python2.7/site-packages/ipykernel/__main__.py:33: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/local/shared/anaconda/envs/yahognn/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


 /gel/rachmaninoff/data/pictures/master/skycam/20141020/180310/stack.exr
Error happened: cannot do a non-empty take from an empty axes.
Processing:  /gel/rachmaninoff/data/pictures/master/skycam/20141030/115226/stack.exr
Ratio: 0.244094848633
Processing:  /gel/rachmaninoff/data/pictures/master/skycam/20131111/122951/stack.exr
Ratio: 0.474945068359
Processing:  /gel/rachmaninoff/data/pictures/master/skycam/20131113/132222/stack.exr
Ratio: 0.176727294922
Processing:  /gel/rachmaninoff/data/pictures/master/skycam/20141021/170735/stack.exr
Ratio: 0.0212020874023
Processing:  /gel/rachmaninoff/data/pictures/master/skycam/20141113/153029/stack.exr
Ratio: 0.175720214844
Processing:  /gel/rachmaninoff/data/pictures/master/skycam/20130823/174809/stack.exr
Ratio: 0.199295043945
Processing:  /gel/rachmaninoff/data/pictures/master/skycam/20141003/173005/stack.exr
Ratio: 0.103561401367
Processing:  /gel/rachmaninoff/data/pictures/master/skycam/20130731/131151/stack.exr
Ratio: 0.425415039062
Process

/home/local/shared/anaconda/envs/yahognn/lib/python2.7/site-packages/numpy/core/_methods.py:71: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
